In [7]:


####### Utilities ############
import numpy as np
import csv
import emoji
from statistics import mean
import re
import collections
from textblob import TextBlob
from warnings import simplefilter
simplefilter(action='ignore', category=Warning)
import spacy
import en_core_web_sm
import nltk 
from nltk.corpus import wordnet 
from  more_itertools import unique_everseen
nltk.download('wordnet')
import pandas as pd

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/shiyuerong/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# Utility functions and supplement data

In [8]:

def convert_label(text):
    label = text.strip().lower()
    if label == 'sentimental strengthening':
        return 0
    elif label == 'sentimental addition':
        return 1
    elif label == 'visual enhancement':
        return 2
    elif label == 'object representation':
        return 3
    elif label == 'object replacement':
        return 4
    elif label == 'content orgnization':
        return 5


def extract_emojis(message: str) -> 'list of emoji':
    return [c for c in message if c in emoji.UNICODE_EMOJI]

emoji_chart = './Emoji_Sentiment_Data_v1.0.csv' # Name of emoji chart

Emoji_type_dict = dict()
input_file = csv.reader(open(emoji_chart,'r'))
for row in input_file:
    Emoji_type_dict[row[0]] = []
    for i in row[1:]:
        Emoji_type_dict[row[0]].append(i)
        
NLP = en_core_web_sm.load()

# Feature generators

In [9]:
def emoji_emotion_discern(message: str) -> list:
    emoji_list = extract_emojis(message) 
    emoji_condition = Emoji_type_dict[emoji_list[0]]
    criteria1 =int(emoji_condition[-1] == 'Emoticons')
    polarity_list = [int(i) for i in emoji_condition[3:6]]
    normalizedPL =[round(i/sum(polarity_list)*100,3) for i in polarity_list] 
    normalizedPL.append(criteria1)
    
    return normalizedPL

def emoji_emotion_discern_no_polarity(message: str) -> list:
    emoji_list = extract_emojis(message) 
    emoji_condition = Emoji_type_dict[emoji_list[0]]
    criteria1 =int(emoji_condition[-1] == 'Emoticons')
    
    return criteria1

def statement_emotion_discern(message: str) -> float:
    return round(TextBlob(message).sentiment.polarity, 3)

def emoji_repeat_discern(message) -> int:
    return len(extract_emojis(message))

def get_position_percentage(message) -> float:
    emojis = extract_emojis(message)
    if not emoji_repeat_discern(message):
        return round(message.find(emojis[0])/len(message)*100,3)
    else: 
        percentage = [p.start()/len(message)*100 for p in re.finditer(emojis[0], message)]
        return round(mean(percentage),3)
    
    
def get_statement_emoji_similarity(message) -> float:
    emoji = extract_emojis(message)[0]
    emoji_def = Emoji_type_dict[emoji][-2].lower()
    message = message.replace(emoji,'')
    token1 = NLP(emoji_def)
    token2 = NLP(message)
    result = []
    for i in token1:
        w1 = wordnet.synsets(i.text)
        for j in token2:
            w2 = wordnet.synsets(j.text)
            for w_1 in w1:
                for w_2 in w2:
                    if w_1 != w_2:
                        score = w_1.wup_similarity(w_2)
                        if score != None: 
                            #print(w_1,w_2)
                            result.append(score)   
            #print(i.text, j.text, i.wup_similarity(j))
            #result.append(i.wup_similarity(j))
    #print(result)
    if len(result) > 0:
        return max(result)
    else:
        return 0

# Training Data generator

In [13]:
import tqdm
FILE_NAME = 'raw_train.txt' # RAW training data
realData = open(FILE_NAME,'r')
file_list = realData.readlines()

label_file = 'raw_label.txt'
realLabel = open(label_file,'r')
label_list = realLabel.readlines()

def get_feature_vec(real_list):
    total_vec = []
    for data in tqdm.tqdm(real_list):
        try:
            c_vec = []
            #t = data.split('\t|\t')[4][1:-1]
            c_vec.extend(emoji_emotion_discern(data))
            c_vec.append(statement_emotion_discern(data))
            c_vec.append(emoji_repeat_discern(data))
            c_vec.append(get_position_percentage(data))
            c_vec.append(get_statement_emoji_similarity(data))
            total_vec.append(c_vec)
        except:
            continue
    return total_vec

data_list = get_feature_vec(file_list)
print(len(data_list))

d = {"x0":[],"x1":[],"x2":[],"x3":[],"x4":[],"x5":[],"x6":[],"x7":[],"y":[]}
for each,label in zip(data_list,label_list):
    d["x0"].append(each[0])   # Percent of this emoji to be used as negativ
    d["x1"].append(each[1])   # Percent of this emoji to be used as neutral
    d["x2"].append(each[2])   # Percent of this emoji to be used as positive
    d["x3"].append(each[3])   # Whether this emoji is ‘Emoticons’ in the chart
    d["x4"].append(each[4])   # TextBlob package to evaluate the polarity score for the statement itself 
    d["x5"].append(each[5])   # Number of times emoji repeated in the message
    d["x6"].append(each[6])   # The position of emoji used in the message from left to right
    d["x7"].append(each[7])   # Similarity score between emoji definition and the whole sentence
    d["y"].append(convert_label(label))    # Label

df = pd.DataFrame.from_dict(d)
df.to_csv('test_data.csv')

100%|██████████| 24/24 [00:02<00:00,  9.62it/s]

24
